In [ ]:
import cv2
import os
import numpy as np
import PIL as pil
from tqdm import tqdm
import pickle

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Drive files

1. Aceptar invitación a la unidad compartida Patrones2022

2. Abrir la unidad, y sobre la carpeta `home`, hacer click derecho y luego "Añadir acceso directo a Drive".

Si eso funcionó, al ejecutar la siguiente celda, debería verse `db_caracteristicas raw_database  trained_models` en el output.

In [16]:
HOME = "/content/drive/My Drive/home/"
!ls "{HOME}"

db_caracteristicas  raw_database  trained_models


In [ ]:
def SavePickleToDrive(data: object, path: str) -> None:
    """
    Save data to drive as pickle
    """
    pick_insert = open("{HOME}/{path}".format(path=path), mode='wb')
    pickle.dump(data, pick_insert)
    pick_insert.close()


def LoadPickleFromDrive(path: str) -> object:
    """
    Load pickle from drive
    """
    pick_load = open("{HOME}/{path}".format(path=path), mode='rb')
    data = pickle.load(pick_load)
    pick_load.close()
    return data

In [ ]:
def LoadImage(path: str, cmap: str = 'gray', echo: bool = True) -> np.ndarray:
    """
    Load an image from a path
    """
    if echo:
        print('Loading image: ' + path)
    img = cv2.imread("{HOME}/{path}".format(path=path), cmap=cmap)
    if echo:
        print("Image size:", img.shape)
    return img


def NofClasses(path: str) -> int:
    """
    Get the classes in a directory
    """
    return len(os.listdir(path)) - 1


def NofSamples(path: str) -> list:
    """
    Get the number of samples for each class
    """
    samples = []
    for subdir in os.listdir(path):
        samples.append(len(os.listdir(path + subdir)))
    return samples


def BuildDataset(path: str, cmap: str = 'gray') -> tuple:
    """
    Build a dataset from a directory, returns a tuple (X, y, #clas, [#sam])
    """
    classes = NofClasses(path)
    samples = NofSamples(path)
    Xsam = np.zeros((sum(samples), classes))
    Ysam = np.zeros((sum(samples), 1))
    i = 0
    ii = 0
    echo = True
    for dir in os.listdir(path):
        for file in tqdm(os.listdir(path + dir)):
            Xsam[ii] = LoadImage(path + dir + '/' + file, cmap=cmap, echo=echo)
            echo = False
            Ysam[ii] = i
            ii += 1
        i += 1
    return (Xsam, Ysam, classes, samples)
    